In [1]:
import functools
import os
import pickle
import random

import numpy as np
import pandas as pd
import scipy.stats as st
import statsmodels.formula.api as smf


@functools.wraps(smf.ols)
def lm(*args, **kwargs):
    return smf.ols(*args, **kwargs).fit()

In [2]:
recommendee = "Fro116"
confidence_interval = 0.95 # x standard deviations
delta_sources = ["item", "user", "related", "maluserrec3"] # TODO get programmatically
cross_validate = True  # if true, train linear model on out of sample data
renormalize_variance_iters = 10

In [3]:
outdir = f"../../data/recommendations/{recommendee}"
os.chdir(outdir)

## Ensemble signals into a linear model

In [4]:
def get_deltas(sources):
    deltas = []
    for source_filename in sources:
        delta = pickle.load(open(source_filename, "rb"))
        source = source_filename.split(".")[0].split("_")[0]
        delta = delta.rename({x: x + f"_{source}" for x in delta.columns}, axis=1)
        deltas.append(delta)
    return pd.concat(deltas, axis=1)

In [5]:
def clean_data(df):
    # fill missing data with reasonable defaults
    delta_sources = [x.split("_")[-1] for x in df.columns if "delta_var" in x]
    for source in delta_sources:
        df.loc[lambda x: x[f"delta_var_{source}"] == np.inf, f"delta_{source}"] = np.nan
        df.loc[
            lambda x: x[f"delta_var_{source}"] == np.inf, f"delta_var_{source}"
        ] = np.nan

        df[f"delta_{source}"] = df[f"delta_{source}"].fillna(0)
        df[f"delta_var_{source}"] = df[f"delta_var_{source}"].fillna(
            df[f"delta_var_{source}"].dropna().median()
        )
    return df

In [6]:
if cross_validate:
    train_df = get_deltas([f"{x}_loocv.pkl" for x in delta_sources])
else:
    train_df = get_deltas([f"{x}.pkl" for x in delta_sources])
delta_corrs = train_df[[f"delta_{source}" for source in delta_sources]].corr()

In [7]:
labelled_data = pickle.load(open("user_anime_list.pkl", "rb"))
labelled_data = clean_data(labelled_data.merge(train_df, on="anime_id", how='left'))

In [8]:
# get model
delta_cols = [f"delta_{source}" for source in delta_sources]
formula = "score ~ " + " + ".join(delta_cols)
model = lm(formula, labelled_data)
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                  score   R-squared:                       0.274
Model:                            OLS   Adj. R-squared:                  0.265
Method:                 Least Squares   F-statistic:                     32.39
Date:                Thu, 27 May 2021   Prob (F-statistic):           6.39e-23
Time:                        02:13:14   Log-Likelihood:                -620.87
No. Observations:                 349   AIC:                             1252.
Df Residuals:                     344   BIC:                             1271.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept             0.1671      0.11

In [9]:
#labelled_data[['score', 'delta_maluserrec', 'delta_item', 'delta_user']].corr()

In [10]:
df = clean_data(get_deltas([f"{x}.pkl" for x in delta_sources]))

In [11]:
blp = pickle.load(open("baseline_predictor.pkl", "rb"))
df["blp"] = blp["blp"]
df["score"] = model.predict(df) + df["blp"]
df["delta"] = df["score"] - df["blp"]

## Compute Confidence Intervals

In [12]:
for _ in range(renormalize_variance_iters):
    for source in delta_sources:
        seen_shows = pickle.load(open("user_anime_list.pkl", "rb"))
        seen_shows = seen_shows.set_index("anime_id")
        seen_shows["delta"] = df[f"delta_{source}"]

        single_delta_model = lm("score ~ delta + 0", seen_shows)

        seen_shows["pred_score"] = single_delta_model.predict(df)
        seen_shows["pred_std"] = np.sqrt(
            (df[f"delta_var_{source}"] + df[f"delta_{source}"] ** 2)
            * (
                single_delta_model.bse["delta"] ** 2
                + single_delta_model.params["delta"] ** 2
            )
            - (df[f"delta_{source}"] ** 2 * single_delta_model.params["delta"] ** 2)
        )
        seen_shows = seen_shows.loc[lambda x: x["pred_std"] < np.inf]

        std_mult = (
            (seen_shows["pred_score"] - seen_shows["score"]) / seen_shows["pred_std"]
        ).std()
        df[f"delta_var_{source}"] *= std_mult ** 2

In [13]:
# compute error bars
model_vars = pd.DataFrame()
for col in delta_cols:
    source = col.split("_")[1]
    model_vars[f"model_delta_var_{source}"] = (
        (df[f"delta_var_{source}"] + df[f"delta_{source}"] ** 2)
        * (model.bse[f"delta_{source}"] ** 2 + model.params[f"delta_{source}"] ** 2)
    ) - df[f"delta_{source}"] ** 2 * model.params[f"delta_{source}"] ** 2
model_stds = np.sqrt(model_vars)

delta_corrs = delta_corrs.loc[lambda x: (x.index.isin(delta_cols)), delta_cols]
delta_variance = np.sum(
    (model_stds.values @ delta_corrs.values) * model_stds.values, axis=1
)
intercept_variance = 0
if "Intercept" in model.bse:
    intercept_variance = model.bse["Intercept"] ** 2
df["std"] = np.sqrt(delta_variance + intercept_variance)

In [14]:
for _ in range(renormalize_variance_iters):
    seen_shows = pickle.load(open("user_anime_list.pkl", "rb"))
    seen_shows = seen_shows.set_index("anime_id")
    seen_shows["score"] += df["blp"]
    seen_shows["pred_score"] = df[f"score"]
    seen_shows["pred_std"] = df["std"]

    seen_shows = seen_shows.loc[lambda x: x["pred_std"] < np.inf]

    std_mult = (
        (seen_shows["pred_score"] - seen_shows["score"]) / seen_shows["pred_std"]
    ).std()
    df["std"] *= std_mult

In [15]:
zscore = st.norm.ppf(1 - (1 - confidence_interval) / 2)
df["score_lower_bound"] = df["score"] - df["std"] * zscore
df["score_upper_bound"] = df["score"] + df["std"] * zscore

## Display Recommendations

In [16]:
anime = pd.read_csv("../../cleaned_data/anime.csv")
anime = anime[["anime_id", "title", "type"]]
df = df.merge(anime, on="anime_id").set_index("anime_id")

In [17]:
# reorder the columns
cols = [
    "title",
    "type",
    "score",
    "score_lower_bound",
    "score_upper_bound",
    "delta",
    "std",
] + delta_cols
df = df[cols + [x for x in df.columns if x not in cols]]

In [18]:
related_series = pickle.load(open("../../processed_data/related_anime_graph.pkl", "rb"))
df = df.merge(related_series, on="anime_id").set_index("anime_id")

In [19]:
new_recs = df.loc[lambda x: ~x.index.isin(labelled_data.anime_id) & (x["type"] == "TV")]

In [20]:
df.sort_values(by="score_lower_bound", ascending=False)[
    :20
]

,title,type,score,score_lower_bound,score_upper_bound,delta,std,delta_item,delta_user,delta_related,delta_maluserrec3,delta_var_item,delta_var_user,delta_var_related,delta_var_maluserrec3,blp,series_id
anime_id,,,,,,,,,,,,,,,,,
32,Neon Genesis Evangelion: The End of Evangelion,Movie,9.470785,7.748607,11.192962,2.189458,0.878678,-0.103332,1.092234,2.285116,1.044580,0.242022,0.416824,1.028549,1.145467,7.281326,20
5114,Fullmetal Alchemist: Brotherhood,TV,9.274007,7.503434,11.044580,1.171124,0.903370,-0.454362,-0.208816,0.000000,2.182521,0.157666,0.204073,1.620177,2.593584,8.102883,2738
11979,Mahou Shoujo Madoka★Magica Movie 2: Eien no Mo...,Movie,9.155442,7.445484,10.865400,1.796650,0.872444,-0.001944,0.240971,2.614022,1.079136,0.195951,0.617167,0.902768,1.401445,7.358792,4026
9756,Mahou Shoujo Madoka★Magica,TV,8.489900,7.360716,9.619083,1.084504,0.576125,-0.245452,0.421659,2.614022,0.136798,0.117351,0.260851,0.902768,0.212365,7.405396,4026
3784,Evangelion: 2.0 You Can (Not) Advance,Movie,9.012478,7.334388,10.690568,1.509456,0.856184,-0.106574,0.571642,2.300928,0.574129,0.234719,0.315450,0.569289,1.811907,7.503022,1701
30,Neon Genesis Evangelion,TV,8.447606,7.151313,9.743898,1.299163,0.661386,-0.062037,0.835813,2.285116,-0.040360,0.221711,0.320320,1.028549,0.188490,7.148442,20
11981,Mahou Shoujo Madoka★Magica Movie 3: Hangyaku n...,Movie,8.688120,7.102386,10.273853,1.321559,0.809063,-0.151492,0.513312,2.614022,0.283841,0.168799,0.484036,0.902768,1.124356,7.366560,4026
2759,Evangelion: 1.0 You Are (Not) Alone,Movie,8.768462,7.042964,10.493960,1.660344,0.880372,-0.021774,0.505513,2.300928,0.764221,0.233925,0.297051,0.569289,2.124862,7.108118,1701
7311,Suzumiya Haruhi no Shoushitsu,Movie,8.159952,7.028723,9.291181,0.385463,0.577168,-0.173952,0.031557,1.380163,-0.073321,0.121310,0.235886,0.598191,0.567034,7.774489,572


In [21]:
epsilon=1e-6
new_recs.loc[lambda x: (x["delta"] > model.params['Intercept'] + epsilon)].sort_values(
    by="score_lower_bound", ascending=False
).groupby("series_id").first().sort_values(by="score_lower_bound", ascending=False)[:50]

,title,type,score,score_lower_bound,score_upper_bound,delta,std,delta_item,delta_user,delta_related,delta_maluserrec3,delta_var_item,delta_var_user,delta_var_related,delta_var_maluserrec3,blp
series_id,,,,,,,,,,,,,,,,
2738,Fullmetal Alchemist: Brotherhood,TV,9.274007,7.503434,11.044580,1.171124,0.903370,-0.454362,-0.208816,0.000000,2.182521,0.157666,0.204073,1.620177,2.593584,8.102883
2690,Steins;Gate 0,TV,8.991758,6.473860,11.509657,1.092721,1.284666,-0.290486,-0.207602,0.988575,1.484362,0.580360,2.778572,1.626520,1.513163,7.899037
4084,Love Live! School Idol Project 2nd Season,TV,7.430610,5.437177,9.424043,0.548810,1.017076,0.161711,-0.112264,0.535378,0.269781,0.603449,1.000104,1.652147,1.210815,6.881801
156,Jigoku Shoujo Futakomori,TV,7.383730,5.406309,9.361152,0.374007,1.008907,-0.586243,-0.254810,0.000000,1.217716,0.297326,1.258492,1.620177,1.313681,7.009724
109,RahXephon,TV,6.878834,5.110444,8.647223,0.445470,0.902256,-0.115282,-0.033752,0.000000,0.568839,0.275476,1.522035,1.620177,0.403559,6.433364
124,Tenkuu no Escaflowne,TV,7.031785,5.071598,8.991972,0.381032,1.000114,-0.064500,0.019650,0.000000,0.364295,0.323571,1.545941,1.620177,0.988815,6.650753
1143,Hidamari Sketch x 365,TV,7.004389,5.055099,8.953679,0.177070,0.994554,0.023257,0.151508,0.000000,-0.163555,0.258761,2.027032,1.620177,0.603533,6.827319
7707,Kono Bijutsubu ni wa Mondai ga Aru!,TV,6.741276,4.982038,8.500514,0.469215,0.897587,0.069243,-0.072971,0.000000,0.434480,0.406927,1.037330,1.620177,0.623003,6.272061
51,Mobile Suit Zeta Gundam,TV,8.210964,4.931934,11.489994,1.342006,1.673005,-0.040197,0.172291,0.652239,1.319441,0.596725,3.788855,0.859624,6.352240,6.868958


In [27]:
# Inreased serendipity!
new_recs.loc[lambda x: (x["delta_user"] > 0)].sort_values(
    by="score_lower_bound", ascending=False
).groupby("series_id").first().sort_values(by="score_lower_bound", ascending=False)[:50]

,title,type,score,score_lower_bound,score_upper_bound,delta,std,delta_item,delta_user,delta_related,delta_maluserrec3,delta_var_item,delta_var_user,delta_var_related,delta_var_maluserrec3,blp
series_id,,,,,,,,,,,,,,,,
1143,Hidamari Sketch x ☆☆☆,TV,7.118493,5.173887,9.063100,0.162624,0.992164,0.026403,0.100608,0.000000,-0.137128,0.276622,2.200340,1.620177,0.410110,6.955870
124,Tenkuu no Escaflowne,TV,7.031785,5.071598,8.991972,0.381032,1.000114,-0.064500,0.019650,0.000000,0.364295,0.323571,1.545941,1.620177,0.988815,6.650753
51,Mobile Suit Zeta Gundam,TV,8.210964,4.931934,11.489994,1.342006,1.673005,-0.040197,0.172291,0.652239,1.319441,0.596725,3.788855,0.859624,6.352240,6.868958
346,Ichigo Mashimaro,TV,6.739902,4.922898,8.556906,0.031217,0.927060,0.053271,0.020005,0.000000,-0.277868,0.301976,1.437257,1.620177,0.604229,6.708685
91,Kimi ga Nozomu Eien,TV,6.510459,4.838508,8.182411,0.190382,0.853052,-0.048550,0.086295,0.000000,0.002163,0.317018,1.050995,1.620177,0.490951,6.320078
5698,Gochuumon wa Usagi desu ka?,TV,6.448211,4.728126,8.168295,0.047648,0.877610,0.039008,0.019304,0.000000,-0.237168,0.422323,1.024228,1.620177,0.489945,6.400562
4084,Love Live! Sunshine!!,TV,6.879511,4.696910,9.062112,0.551609,1.113592,0.305470,0.029907,0.535378,-0.030781,0.617232,1.989479,1.652147,0.819367,6.327902
7657,Amanchu!,TV,6.695332,4.657520,8.733145,0.459486,1.039720,-0.043518,0.233290,0.000000,0.240724,0.237943,2.141247,1.620177,0.879691,6.235846
427,Mugen no Ryvius,TV,7.195680,4.611560,9.779799,0.682309,1.318453,0.142223,0.401186,0.000000,0.187121,0.414414,4.304327,1.620177,0.907573,6.513371
